In [1]:
print('get all txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import google_bq_utils as bqu
sys.path.pop()

import time
import datetime

import numpy as np
import pandas as pd
import os

get all txs


In [2]:
col_list = [
        'dt','blockchain','name','layer','chain_id'
        , 'num_raw_txs', 'num_success_txs','num_qualified_txs','source'
        ]
query_name = 'dune_all_txs'
rerun_hrs = 4

In [3]:
chain_config = [
    #blockchain, display_name, count_func, gas_field, transactions_table
    ['bitcoin','Bitcoin','count(*)','fee','transactions'],
    ['near','Near','count(distinct tx_hash)','gas_price','actions'],
    ['aptos','Aptos','count(*)','gas_used','transactions'],
    ['stellar','Stellas','count(*)','fee_charged','history_transactions'],
]

In [4]:
trailing_days = 365#180

In [5]:
days_param = d.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')

In [6]:
# Run Dune
print('     dune runs')
fees_df = d.get_dune_data(query_id = 4229341, #https://dune.com/queries/4229341
    name = query_name,
    path = "outputs",
    performance="large",
    params = [days_param],
    num_hours_to_rerun=rerun_hrs
)

     dune runs
Results available at https://dune.com/queries/4229341?trailing_days=365
getting latest result


2024-11-01 10:49:53,059 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2024-11-01 10:49:53,060 INFO dune_client.api.base executing 4229341 on large cluster
2024-11-01 10:49:53,469 INFO dune_client.api.base waiting for query execution 01JBM38HTMAVHD51G4WGC2E22Y to complete: ExecutionState.PENDING (queue position: 1)
2024-11-01 10:49:54,627 INFO dune_client.api.base waiting for query execution 01JBM38HTMAVHD51G4WGC2E22Y to complete: ExecutionState.PENDING (queue position: 1)
2024-11-01 10:49:55,787 INFO dune_client.api.base waiting for query execution 01JBM38HTMAVHD51G4WGC2E22Y to complete: ExecutionState.PENDING (queue position: 1)
2024-11-01 10:49:56,941 INFO dune_client.api.base waiting for query execution 01JBM38HTMAVHD51G4WGC2E22Y to complete: ExecutionState.EXECUTING
2024-11-01 10:49:58,098 INFO dune_client.api.base waiting for query execution 01JBM38HTMAV

In [7]:
fees_df.sample(5)

,blockchain,chain_id,display_name,dt,num_txs,tx_fee_currency,tx_fee_usd,last_updated
2265,zora,7777777.0,ZORA,2024-09-13,92011,0x0000000000000000000000000000000000000000,1.676516e+02,2024-11-01 14:49:53.237807
3979,celo,42220.0,Celo,2023-12-11,2,0xe8537a3d056da446677b9e9d6c5db704eaab4787,3.576296e-03,2024-11-01 14:49:53.237807
3624,zora,7777777.0,ZORA,2024-04-14,626206,0x0000000000000000000000000000000000000000,1.328617e+03,2024-11-01 14:49:53.237807
4711,zkevm,1101.0,Polygon zkEVM,2024-03-15,9398,0x0000000000000000000000000000000000000000,2.835192e+04,2024-11-01 14:49:53.237807
4762,tron,728126428.0,TRON,2024-07-14,1958424,0x0000000000000000000000000000000000000000,4.561904e+06,2024-11-01 14:49:53.237807


In [8]:
unique_blockchains = fees_df['blockchain'].unique().tolist()

In [ ]:
chain_dfs = []
for row in chain_config:
    blockchain = row[0]
    print(blockchain)
    if blockchain in unique_blockchains:
        continue
    else:
        #blockchain, display_name, count_func, gas_field, transactions_table
        blockchain_param = d.generate_query_parameter(input=blockchain,field_name='blockchain',dtype='text')
        display_name_param = d.generate_query_parameter(input=row[1],field_name='display_name',dtype='text')
        count_func_param = d.generate_query_parameter(input=row[2],field_name='count_func',dtype='text')
        gas_field_param = d.generate_query_parameter(input=row[3],field_name='gas_field',dtype='text')
        transactions_table_param = d.generate_query_parameter(input=row[4],field_name='transactions_table',dtype='text')

        chain_df = d.get_dune_data(query_id = 4230061, #https://dune.com/queries/4230061
            name = query_name + '_by_chain',
            path = "outputs",
            performance="large",
            params = [days_param,blockchain_param,display_name_param,count_func_param,gas_field_param,transactions_table_param],
            num_hours_to_rerun=rerun_hrs
        )
        print(chain_df.sample(3))
        chain_dfs.append(chain_df)

chain_df_agg = pd.concat(chain_dfs)

bitcoin
Results available at https://dune.com/queries/4230061?trailing_days=365&blockchain=bitcoin&display_name=Bitcoin&count_func=count%28%2A%29&gas_field=fee&transactions_table=transactions
getting latest result


2024-11-01 10:54:00,954 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2024-11-01 10:54:00,955 INFO dune_client.api.base executing 4230061 on large cluster
2024-11-01 10:54:01,266 INFO dune_client.api.base waiting for query execution 01JBM3G3XH4CQR660SAEQR6Z8D to complete: ExecutionState.PENDING (queue position: 1)
2024-11-01 10:54:02,435 INFO dune_client.api.base waiting for query execution 01JBM3G3XH4CQR660SAEQR6Z8D to complete: ExecutionState.EXECUTING
2024-11-01 10:54:03,628 INFO dune_client.api.base waiting for query execution 01JBM3G3XH4CQR660SAEQR6Z8D to complete: ExecutionState.EXECUTING
2024-11-01 10:54:04,780 INFO dune_client.api.base waiting for query execution 01JBM3G3XH4CQR660SAEQR6Z8D to complete: ExecutionState.EXECUTING
2024-11-01 10:54:05,988 INFO dune_client.api.base waiting for query execution 01JBM3G3XH4CQR660SAEQR6Z8D to complete: Executio

    blockchain chain_id display_name          dt  num_txs tx_fee_currency  \
52     bitcoin     None      Bitcoin  2024-08-01   544289            None   
1      bitcoin     None      Bitcoin  2024-04-09   453653            None   
193    bitcoin     None      Bitcoin  2024-10-18   474143            None   

    tx_fee_native tx_fee_usd               last_updated  
52           None       None 2024-11-01 14:54:01.138159  
1            None       None 2024-11-01 14:54:01.138159  
193          None       None 2024-11-01 14:54:01.138159  
near
Results available at https://dune.com/queries/4230061?trailing_days=365&blockchain=near&display_name=Near&count_func=count%28distinct+tx_hash%29&gas_field=gas_price&transactions_table=actions
getting latest result


2024-11-01 10:54:37,474 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2024-11-01 10:54:37,476 INFO dune_client.api.base executing 4230061 on large cluster
2024-11-01 10:54:37,827 INFO dune_client.api.base waiting for query execution 01JBM3H7JVA4GT5TQWKG9BGKTD to complete: ExecutionState.PENDING (queue position: 1)
2024-11-01 10:54:39,034 INFO dune_client.api.base waiting for query execution 01JBM3H7JVA4GT5TQWKG9BGKTD to complete: ExecutionState.PENDING (queue position: 1)
2024-11-01 10:54:40,234 INFO dune_client.api.base waiting for query execution 01JBM3H7JVA4GT5TQWKG9BGKTD to complete: ExecutionState.EXECUTING
2024-11-01 10:54:41,515 INFO dune_client.api.base waiting for query execution 01JBM3H7JVA4GT5TQWKG9BGKTD to complete: ExecutionState.EXECUTING
2024-11-01 10:54:42,663 INFO dune_client.api.base waiting for query execution 01JBM3H7JVA4GT5TQWKG9BGKTD to 

In [ ]:
print(fees_df.columns)
print(chain_df_agg.columns)

In [ ]:
dune_df = pd.concat([fees_df,chain_df_agg])

In [12]:
if not dune_df.empty:
    dune_df['source'] = 'dune'
    dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)
    dune_df = dune_df[dune_df['dt'].dt.date < datetime.datetime.now().date()]
    dune_df['chain_id'] = dune_df['chain_id'].astype(str)
    dune_df['chain_id'] = dune_df['chain_id'].astype(str).str.replace(r'\.0$', '', regex=True)

In [ ]:
# Verify that all elements are strings
assert dune_df['chain_id'].apply(type).eq(str).all(), "Not all elements are strings"
print(dune_df['chain_id'].dtype)

In [ ]:
# dune_df.sample(5)
print(dune_df.dtypes)

In [ ]:
print(f"dune_df shape: {dune_df.shape}")

In [ ]:
# Identify duplicate combinations
duplicates = dune_df.duplicated(subset=['blockchain', 'dt'], keep=False)

# View the duplicate rows
duplicate_rows = dune_df[duplicates]

# Display the duplicate rows
print(duplicate_rows)

# Get a count of duplicates for each combination
duplicate_counts = dune_df.groupby(['blockchain', 'dt']).size().reset_index(name='count')
duplicate_counts = duplicate_counts[duplicate_counts['count'] > 1]

print("\nDuplicate combination counts:")
print(duplicate_counts)

In [ ]:
#BQ Upload
bqu.append_and_upsert_df_to_bq_table(dune_df, query_name, unique_keys = ['blockchain','dt','tx_fee_currency'])